# Backtesting A Trading Strategy Part 2

## How To Retrieve S&P Constituents Historical Data Using Python

## Introduction
Backtesting is a tool to measure the performance of a trading strategy using historical data. The backtesting process consists of three parts: 1. determining the universe of securities where we will invest in (e.g. equity or fixed income? US or emerging markets?); 2. gathering historical data for the universe of securities; and 3. implementing a trading strategy using the historical data collected.  

In the previous article, I illustrated the first step in the backtesting process of determining the universe of stocks, namely the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices. In this article, I will discuss the second step of the backtesting process of collecting historical data for each constituent of the universe of stocks. 

## Retrieving S&P Constituents Historical Data
### Step By Step
1. Load the S&P tickers which were gathered from the previous article. 
2. Collect the S&P constituents' 5-year historical data using Python package pandas-datareader from the Investors Exchange (IEX). 

You can find the code below on https://github.com/DinodC/backtesting-trading-strategy.

Import packages

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import pickle
import pandas_datareader.data as web

### S&P Constituents Tickers
In this section, we load the lists pickled from the last article.

Set an id for each index

In [2]:
id = ['sp500', 'sp400', 'sp600']

Create a dictionary to map each id to a tickers file

In [3]:
input_file = {'sp500': 'sp500_barchart.pickle',
              'sp400': 'sp400_barchart.pickle', 
              'sp600': 'sp600_barchart.pickle'} 

Define a dictionary to map each id to a tickers list

In [4]:
sp500_tickers = []
sp400_tickers = []
sp600_tickers = []
sp_tickers = {'sp500': sp500_tickers,
              'sp400': sp400_tickers,
              'sp600': sp600_tickers}

Fill the tickers lists

In [5]:
for i in input_file:
    with open(input_file[i], 'rb') as f:
        
        # Update tickers list        
        sp_tickers[i] = pickle.load(f)

        # Sort tickers list
        sp_tickers[i].sort()
        
    f.close()

### S&P Constituents Historical Data 

Define dictionary of historical data

In [6]:
sp500_data = pd.DataFrame()
sp400_data = pd.DataFrame()
sp600_data = pd.DataFrame()
sp_data = {'sp500': sp500_data,
           'sp400': sp400_data,
           'sp600': sp600_data}

Set the start and date of the historical data

In [7]:
start_date = '2014-01-01'
end_date = '2020-01-01'

Set the source Investors Exchange(IEX) to be used

In [8]:
source = 'iex'

Create a dictionary to map each id to an output file

In [9]:
output_file = {'sp500': 'sp500_data.pickle',
               'sp400': 'sp400_data.pickle',
               'sp600': 'sp600_data.pickle'}

Retrieve historical data for each constituent of each S&P index

In [10]:
for i in output_file:
    
    # Retrieve historical data 
    # Note that we set number of tickers to < 100 because DataReader gives error when number of tickers > 100
    data1 = web.DataReader(sp_tickers[i][:98], source, start_date, end_date)
    data2 = web.DataReader(sp_tickers[i][98:198], source, start_date, end_date)
    data3 = web.DataReader(sp_tickers[i][198:298], source, start_date, end_date)
    data4 = web.DataReader(sp_tickers[i][298:398], source, start_date, end_date)
    data5 = web.DataReader(sp_tickers[i][398:498], source, start_date, end_date)
    if i == 'sp400':
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5], axis=1, sort=True)
    if i == 'sp500':
        data6 = web.DataReader(sp_tickers[i][498:], source, start_date, end_date)
        # Concatenate historical data        
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6], axis=1, sort=True)
    elif i == 'sp600':
        data6 = web.DataReader(sp_tickers[i][498:598], source, start_date, end_date)
        data7 = web.DataReader(sp_tickers[i][598:], source, start_date, end_date)
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6, data7], axis=1, sort=True)        
    else:
        pass
            
    # Convert index to datetime
    sp_data[i].index = pd.to_datetime(sp_data[i].index)
    
    # Save historical data to file
    with open(output_file[i], 'wb') as f:
        pickle.dump(sp_data[i], f)
    f.close()

## Constituents Close Prices

### S&P 500 Index

Look at the dimensions of our DataFrame

In [11]:
sp_data['sp500'].close.shape

(1258, 505)

Check the first rows

In [12]:
sp_data['sp500'].close.head()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2014-06-11,40.1199,40.2868,125.0694,86.0249,45.0769,66.4223,22.92,35.9633,74.9719,67.46,...,25.6691,40.9150,84.1321,46.6073,28.8084,35.4555,51.7899,101.5068,28.0164,30.9798
2014-06-12,39.7726,38.2958,123.2252,84.5860,44.6031,65.9975,23.03,35.7925,74.5018,66.56,...,25.8547,41.1019,83.8928,46.4230,28.5372,35.2780,51.2950,101.0367,27.7535,30.8448
2014-06-13,39.8407,38.4672,123.7407,83.6603,45.0187,66.2930,22.95,35.7745,74.7820,66.82,...,25.8884,41.6091,84.7098,46.3744,28.4920,36.0532,51.5945,101.2861,27.8192,30.9702
2014-06-16,39.7181,39.1150,124.0086,84.5035,44.8857,66.0529,23.27,35.8734,74.8634,67.62,...,26.0740,41.9028,84.9326,46.4424,28.3791,35.9318,51.5099,100.7105,27.4060,30.9798
2014-06-17,40.0927,39.8867,124.9411,84.3935,45.1351,66.2561,23.43,35.8375,74.5832,67.54,...,26.0910,42.2409,84.5200,46.2677,28.8310,36.0346,51.7639,100.4707,27.9601,31.6355


In [13]:
sp_data['sp500'].close[['A', 'AAL', 'AAP', 'AAPL', 'ABBV']].head()

Symbols,A,AAL,AAP,AAPL,ABBV
date,,,,,
2014-06-11,40.1199,40.2868,125.0694,86.0249,45.0769
2014-06-12,39.7726,38.2958,123.2252,84.5860,44.6031
2014-06-13,39.8407,38.4672,123.7407,83.6603,45.0187
2014-06-16,39.7181,39.1150,124.0086,84.5035,44.8857
2014-06-17,40.0927,39.8867,124.9411,84.3935,45.1351


Check the end rows

In [14]:
sp_data['sp500'].close.tail()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2019-06-04,67.95,29.12,154.61,179.64,76.75,82.21,267.06,77.46,177.97,268.71,...,57.78,106.90,73.59,54.40,33.30,77.40,106.97,117.41,44.40,108.12
2019-06-05,68.35,30.36,154.61,182.54,77.06,81.65,268.80,78.69,179.56,272.86,...,59.32,105.60,72.98,55.38,33.42,78.88,107.29,118.54,44.18,108.50
2019-06-06,69.16,30.38,154.90,185.22,77.07,81.75,269.19,80.09,180.40,274.80,...,59.80,106.01,74.31,55.63,34.03,79.15,108.42,120.31,44.24,108.89
2019-06-07,69.52,30.92,155.35,190.15,77.43,83.48,267.87,80.74,182.92,278.16,...,59.43,107.49,74.58,55.94,34.16,79.56,109.07,120.73,43.64,110.06
2019-06-10,70.29,30.76,153.52,192.58,76.95,84.77,272.43,81.27,184.44,280.34,...,59.26,110.88,74.91,57.10,34.69,80.38,108.65,121.71,43.84,110.22


In [15]:
sp_data['sp500'].close[['A', 'AAL', 'AAP', 'AAPL', 'ABBV']].tail()

Symbols,A,AAL,AAP,AAPL,ABBV
date,,,,,
2019-06-04,67.95,29.12,154.61,179.64,76.75
2019-06-05,68.35,30.36,154.61,182.54,77.06
2019-06-06,69.16,30.38,154.90,185.22,77.07
2019-06-07,69.52,30.92,155.35,190.15,77.43
2019-06-10,70.29,30.76,153.52,192.58,76.95


Descriptive stats

In [16]:
sp_data['sp500'].close.describe()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,52.010924,41.207727,145.418612,135.094399,66.653293,84.553883,162.741109,49.113396,118.473403,142.243243,...,39.351443,58.850843,75.442714,53.174206,26.877544,51.296575,66.602743,111.697803,36.704361,59.798250
std,13.866577,6.366236,24.128339,38.127616,17.721243,9.483389,116.577717,12.653128,31.189929,70.413470,...,8.322974,22.240068,4.717698,7.833835,3.222176,16.349757,15.997628,9.929764,10.736900,20.223068
min,32.258600,24.539800,79.168700,82.743800,42.066600,65.718100,22.220000,33.935700,68.852300,60.880000,...,25.247700,32.502600,58.967500,34.178400,18.532600,28.874500,44.181800,89.236100,18.885300,30.652000
25%,39.393500,36.586125,132.519725,103.622650,53.170800,77.738125,80.662500,39.405975,92.616475,82.092500,...,31.485800,42.102950,72.534125,48.731800,24.215550,35.033175,53.060925,103.343675,26.826025,44.749325
50%,46.097600,40.843300,150.457700,119.473800,58.470800,83.840300,118.550000,43.040600,112.097050,107.945000,...,39.055650,52.368850,75.817000,54.521200,26.573400,48.085300,61.564200,112.761500,38.198900,51.333600
75%,65.591850,46.119900,161.899875,167.841200,82.894300,89.847025,261.935000,58.171250,149.591750,212.247500,...,45.444750,68.789375,78.491000,59.650550,29.650500,67.341800,80.202225,119.131625,46.431025,80.809150
max,81.940000,57.586600,199.159900,229.392000,116.445400,107.649700,449.750000,81.270000,184.440000,289.250000,...,59.800000,139.263300,86.137400,67.795300,35.000000,83.549000,109.070000,130.912800,57.139500,110.220000


In [17]:
sp_data['sp500'].close[['A', 'AAL', 'AAP', 'AAPL', 'ABBV']].describe()

Symbols,A,AAL,AAP,AAPL,ABBV
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,52.010924,41.207727,145.418612,135.094399,66.653293
std,13.866577,6.366236,24.128339,38.127616,17.721243
min,32.258600,24.539800,79.168700,82.743800,42.066600
25%,39.393500,36.586125,132.519725,103.622650,53.170800
50%,46.097600,40.843300,150.457700,119.473800,58.470800
75%,65.591850,46.119900,161.899875,167.841200,82.894300
max,81.940000,57.586600,199.159900,229.392000,116.445400


### S&P MidCap 400 Index

Look at the dimensions of our DataFrame

In [18]:
sp_data['sp400'].close.shape

(1297, 400)

Check the first rows

In [19]:
sp_data['sp400'].close.head()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2014-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
sp_data['sp400'].close[['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM']].head()

Symbols,AAN,ACC,ACHC,ACIW,ACM
2014-05-07,NaN,NaN,NaN,NaN,NaN
2014-05-08,NaN,NaN,NaN,NaN,NaN
2014-05-09,NaN,NaN,NaN,NaN,NaN
2014-05-12,NaN,NaN,NaN,NaN,NaN
2014-05-13,NaN,NaN,NaN,NaN,NaN


Check the end rows

In [21]:
sp_data['sp400'].close.tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2019-06-04,54.88,46.17,33.72,32.10,33.17,18.92,18.52,101.46,69.42,82.76,...,39.82,18.90,111.80,73.10,41.11,13.29,55.28,683.58,31.79,177.08
2019-06-05,54.97,46.97,33.30,32.26,33.29,19.39,18.54,101.80,69.96,84.89,...,40.63,19.04,113.86,73.68,41.08,13.09,53.93,693.19,31.69,181.59
2019-06-06,54.72,47.02,33.20,32.40,33.18,19.75,17.58,101.53,69.16,85.26,...,40.88,18.46,114.32,73.48,41.01,13.21,53.32,692.20,31.57,181.76
2019-06-07,55.72,47.17,33.91,32.34,33.47,20.39,17.50,100.88,69.75,84.99,...,40.84,18.95,113.60,73.28,41.78,13.58,53.78,690.74,31.48,188.37
2019-06-10,59.23,46.81,33.81,32.75,33.45,20.79,16.76,100.83,70.11,84.70,...,40.63,19.40,113.63,71.51,42.55,13.80,55.55,684.74,31.95,194.51


In [22]:
sp_data['sp400'].close[['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM']].tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM
2019-06-04,54.88,46.17,33.72,32.10,33.17
2019-06-05,54.97,46.97,33.30,32.26,33.29
2019-06-06,54.72,47.02,33.20,32.40,33.18
2019-06-07,55.72,47.17,33.91,32.34,33.47
2019-06-10,59.23,46.81,33.81,32.75,33.45


Descriptive stats

In [23]:
sp_data['sp400'].close.describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,655.00000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,35.344696,39.074002,48.806781,22.656060,32.409610,51.41140,15.791684,76.790031,55.704826,59.509322,...,29.638305,29.788041,63.744381,31.766971,37.016177,24.529339,54.093362,538.956165,40.046097,106.230811
std,9.825081,4.946453,13.903160,4.065761,3.178291,20.70094,3.997170,20.795256,9.501151,13.608161,...,4.889501,23.460312,16.066755,25.666425,7.006602,8.798477,25.570709,71.983064,13.133896,42.282714
min,20.118600,27.793900,24.750000,13.617500,23.150000,12.57000,8.872200,46.029500,40.217700,37.847600,...,20.625100,3.780000,38.906800,9.233700,25.207900,6.455100,19.560000,405.789900,15.230000,46.930000
25%,25.957675,34.718300,38.355000,19.470000,30.370000,31.95000,12.927300,58.436225,46.935500,45.647675,...,24.419950,12.060000,48.823450,15.752725,31.123075,18.589200,34.322500,473.883600,31.690000,74.915000
50%,34.792300,39.727900,46.525000,22.275000,32.465000,57.04690,14.696200,75.144950,54.295400,58.636750,...,29.886900,20.890000,66.107300,19.371300,34.954450,23.753650,46.485000,545.605200,38.850000,91.865000
75%,43.259125,43.422450,59.717500,24.240000,34.537500,66.54120,18.327025,96.480175,63.707075,73.222850,...,33.454425,44.240000,75.237300,36.257025,42.975150,32.198000,64.747500,601.316050,45.397500,136.300000
max,59.230000,48.353300,82.970000,35.520000,40.130000,84.04680,28.421700,113.252900,74.760300,85.260000,...,40.880000,103.090000,114.320000,99.250000,54.884400,45.545800,114.540000,693.190000,84.960000,235.440000


In [24]:
sp_data['sp400'].close[['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM']].describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,35.344696,39.074002,48.806781,22.656060,32.409610
std,9.825081,4.946453,13.903160,4.065761,3.178291
min,20.118600,27.793900,24.750000,13.617500,23.150000
25%,25.957675,34.718300,38.355000,19.470000,30.370000
50%,34.792300,39.727900,46.525000,22.275000,32.465000
75%,43.259125,43.422450,59.717500,24.240000,34.537500
max,59.230000,48.353300,82.970000,35.520000,40.130000


## S&P SmallCap 600 Index

Look at the dimensions of our DataFrame

In [25]:
sp_data['sp600'].close.shape

(1259, 601)

Check the first rows

In [26]:
sp_data['sp600'].close.head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
2014-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-11,22.33,20.7695,30.1070,38.07,13.91,20.9537,64.62,24.3261,NaN,7.68,...,10.8245,21.2765,76.44,9.3183,57.5835,24.8378,NaN,19.4619,23.5042,28.99
2014-06-12,22.30,20.5904,29.9566,37.05,14.03,20.7716,62.44,24.1543,NaN,7.60,...,10.7626,21.3093,76.34,9.2923,57.2527,24.3284,NaN,19.2547,23.0351,28.27
2014-06-13,21.86,20.3792,29.9654,37.29,13.87,20.6183,62.28,23.9282,NaN,7.64,...,10.7795,21.1865,76.81,9.3053,57.3094,24.8095,NaN,19.5742,23.2012,28.15
2014-06-16,22.71,20.5712,30.0273,36.95,13.78,20.3980,63.14,23.7745,NaN,7.76,...,10.8864,20.8754,78.29,9.2727,57.4985,24.6302,NaN,19.5828,22.8885,28.21


In [27]:
sp_data['sp600'].close[['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN']].head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN
2014-06-10,NaN,NaN,NaN,NaN,NaN
2014-06-11,22.33,20.7695,30.1070,38.07,13.91
2014-06-12,22.30,20.5904,29.9566,37.05,14.03
2014-06-13,21.86,20.3792,29.9654,37.29,13.87
2014-06-16,22.71,20.5712,30.0273,36.95,13.78


Check the end rows

In [28]:
sp_data['sp600'].close.tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
2019-06-04,9.19,47.80,45.11,39.70,67.63,37.25,79.67,37.60,35.52,15.30,...,4.08,27.03,137.79,12.54,85.75,28.42,20.95,21.32,13.16,20.65
2019-06-05,9.23,48.08,46.39,38.62,68.00,37.03,77.80,37.28,36.20,14.95,...,4.05,27.46,137.16,12.70,85.79,28.25,21.00,20.70,12.86,19.94
2019-06-06,9.30,48.18,46.39,37.99,67.97,36.90,76.95,39.78,37.06,14.89,...,3.99,27.52,136.06,12.67,86.42,28.32,20.85,20.69,13.01,18.67
2019-06-07,9.50,48.85,46.61,39.88,69.18,36.58,77.95,39.36,38.34,15.03,...,4.05,27.47,136.72,12.68,87.35,28.50,21.01,20.93,12.98,21.65
2019-06-10,9.60,48.64,46.51,40.30,71.91,37.20,78.88,39.47,38.69,15.74,...,4.21,27.29,142.45,12.50,87.97,28.39,21.12,20.57,13.19,21.46


In [29]:
sp_data['sp600'].close[['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN']].tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN
2019-06-04,9.19,47.80,45.11,39.70,67.63
2019-06-05,9.23,48.08,46.39,38.62,68.00
2019-06-06,9.30,48.18,46.39,37.99,67.97
2019-06-07,9.50,48.85,46.61,39.88,69.18
2019-06-10,9.60,48.64,46.51,40.30,71.91


Descriptive stats

In [30]:
sp_data['sp600'].close.describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,152.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1094.000000,1258.000000,1258.000000,1258.000000
mean,26.523188,29.595272,37.383077,49.431387,31.595862,36.468555,67.475747,32.757237,30.553658,16.014777,...,7.052403,26.183249,75.539273,10.755358,63.426474,26.717355,17.466643,26.235295,18.697389,22.802699
std,17.116913,7.763096,3.595458,10.156758,15.917380,10.508064,10.126593,5.585728,3.460363,6.525089,...,1.793529,3.155567,28.283319,1.387724,10.670268,5.752016,3.218554,6.633022,4.463565,6.799962
min,8.380000,16.287400,29.352300,31.400000,10.500000,20.120100,45.070000,22.347500,20.930200,6.840000,...,3.990000,20.496900,26.700000,7.337200,43.206400,14.741000,10.508700,11.940500,8.155900,11.450000
25%,14.810000,22.305875,35.384150,40.970000,22.285000,26.449975,59.167500,28.549550,28.943125,10.600000,...,5.664050,23.158525,50.137500,9.794800,54.413025,23.000950,14.932475,21.145625,16.119775,17.500000
50%,19.845000,30.684600,37.508550,49.150000,25.175000,35.079700,67.750000,31.390800,30.315400,13.925000,...,6.576000,26.576000,76.820000,10.795250,61.023300,26.738850,17.782900,26.661550,19.138750,21.850000
75%,33.927500,35.473400,39.254350,56.542500,38.340000,45.666850,74.352500,37.597450,32.236275,20.950000,...,8.228400,29.164850,101.162500,11.707950,72.812975,30.402625,19.650875,30.475175,22.118050,26.700000
max,99.610000,51.631400,46.770000,74.000000,74.890000,57.403000,95.540000,43.220900,38.690000,36.625000,...,11.148200,31.602900,142.450000,14.108200,87.970000,39.469500,24.438200,41.762300,30.738900,41.400000


In [31]:
sp_data['sp600'].close[['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN']].describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,26.523188,29.595272,37.383077,49.431387,31.595862
std,17.116913,7.763096,3.595458,10.156758,15.917380
min,8.380000,16.287400,29.352300,31.400000,10.500000
25%,14.810000,22.305875,35.384150,40.970000,22.285000
50%,19.845000,30.684600,37.508550,49.150000,25.175000
75%,33.927500,35.473400,39.254350,56.542500,38.340000
max,99.610000,51.631400,46.770000,74.000000,74.890000


## Summary
In this article, we retrieved historical data for every constituent in our universe of stocks - the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices. The 5-year historical data is relatively straightforward to obtain, and is provided for free by the Investors Exchange. In the next article, we implement a simple trading strategy, and backtest it using the historical data collected.